# Generate prompts from raw stimuli and instructions for different measurements
Author: Hening Wang
Date last modified: 30.08.2023

TODO: Add some descriptions about the document


In [9]:
# Import relevant modules
import pandas as pd
import numpy as np


## Encoding and pre-processing relevant information from raw stimuli

TODO: Add descritions about the context of each files in raw_stimuli

In [10]:
# Reading the content of options into a DataFrame
content_options = pd.read_csv('raw_stimuli/MaximsOptions.txt', sep='\t', names=['item_number', 'option_number', 'option_text'])

# Create a list of lists of options for each item
options = []
for item_number in content_options.item_number.unique():
    options.append(content_options[content_options.item_number == item_number].option_text.tolist())

# Read key from the file
key_options = pd.read_csv('raw_stimuli/MaximsKeySimple.txt', sep='\t', names=["phenomenon",'item_number', "option_number",'content'])

# Find the key for each item
keys_items_pairs = []
for i, row in key_options.iterrows():
    if row.content == 'Correct':
        keys_items_pairs.append((row.item_number, row.option_number))

# Arrange the keys_items_pairs in the upscent order of item_number
keys_items_pairs.sort(key=lambda x: x[0])

# Reading the scenario content from the file
scenarios = pd.read_csv('raw_stimuli/MaximsScenarios.txt', sep='\t', names=['item_number', 'scenario'])


# Get true answer with key item pairs and option content
true_answer = []
for item_number, option_number in keys_items_pairs:
    matching_option = content_options[
        (content_options['item_number'] == item_number) & 
        (content_options['option_number'] == option_number)
    ]['option_text']
    true_answer.extend(matching_option.tolist())

print(true_answer)

['He disliked the talk that his boss gave but does not want to criticize his boss.  ', "Alex thinks that Richard's painting is mediocre.", 'She does not want to discuss the topic that Leslie has raised.', "He pretends that he did not understand his wife's comment because he wants to watch the game.", "Mark's opinion of Jane as a financial manager is very low.", "Lucy's mom does not know where exactly Vincent lives.", 'The food at the wedding was mediocre.', 'Mary is irritated with Ken.', 'Cindy does not know what type of shoes she wants to buy.', 'He does not want to talk about school with his mom.', "Mark's wife has little experience with cooking.", "Marcie thinks that her husband's expectations about breakfast are too high.", 'Paul thinks that the actress cannot sing.', 'Melissa thinks that statistical information is useless in this debate.', 'He thinks that Rosy has no reason to believe that the strangers are nice.', 'Freddie will never give Berta any money.', 'Katy doubts that Mr. 

## Generate prompts for Rating task

TODO: 
1. What is a rating task
2. We use three different prompts for rating task, plausible, possible, likely. 

In [15]:
# Read the instructions of rating
scales = ["plausible", "appropriate", "possible", "likely"]

for scale in scales:
    file_path_instructions_rating = f'prompt_rating/MaximsInstructions_Rating_{scale}.txt'
    with open(file_path_instructions_rating, 'r') as file:
        instructions_rating = file.read()

# Combine instructions, scenarios and true anwsers together
    new_prompts = []
    for i in range(len(true_answer)):
        new_prompts.append(instructions_rating + " Scenario: " + scenarios.scenario[i] + " Explanation: " + true_answer[i])
    print(new_prompts)
    # Create a new DataFrame for the new prompts for rating task
    dict_prompts_rating = {'item_number': scenarios.item_number,
                        'prompt': new_prompts,
                        'true_answer_key': [keys_items_pair[1] for keys_items_pair in keys_items_pairs],
                        'true_answer': true_answer,
                        'scenarios': scenarios.scenario,
                        'options': options}

    df_prompts_rating = pd.DataFrame(dict_prompts_rating)
    df_prompts_rating.to_csv(f"prompt_rating/Maxims_prompts_Rating_{scale}.csv", index=False)




['Task: You will read short stories that describe everyday situations. Each story will be followed by an explanation of why one character in that scenario responds in a certain way. Read each story and the associated explanation. Your task is to decide how plausible is that explanation. The degree varies from "very implausible", "implausible", "neutral", "plausible", "very plausible". Answer only with one degree. Scenario: Bob is having a lunch meeting with his boss, Mr. James. Mr. James asks Bob: "Did you like the presentation that I gave at the board meeting yesterday?" Bob responds: "I cannot wait for our trip to Japan next week." Why has Bob responded like this?\r Explanation: He disliked the talk that his boss gave but does not want to criticize his boss.  ', 'Task: You will read short stories that describe everyday situations. Each story will be followed by an explanation of why one character in that scenario responds in a certain way. Read each story and the associated explanati

In [13]:
print(df_prompts_rating.prompt[1])

 Explanation: Alex thinks that Richard's painting is mediocre.tions. Each story will be followed by an explanation of why one character in that scenario responds in a certain way. Read each story and the associated explanation. Your task is to decide how plausible is that explanation. The degree varies from "very unlikely", "unlikely", "neutral", "likely", "very likely". Answer only with one degree. Scenario: Richard came to visit his friend, Alex. Alex is an artist. Richard brought one of his artworks to Alex and asked Alex: "What do you think of my painting?" Alex responds, "Have you noticed that I painted the walls?" Why has Alex responded like this?


In [3]:
# Combine instructions, scenarios and true anwsers together
new_prompts = []
for i in range(len(true_answer)):
    new_prompts.append(instructions_rating + " " + "Scenario: " + scenarios.scenario[i] + " " + true_answer[i])

# Create a new DataFrame for the new prompts for rating task
dict_prompts_rating = {'item_number': scenarios.item_number,
                       'prompt': new_prompts,
                       'true_answer_key': [keys_items_pair[1] for keys_items_pair in keys_items_pairs],
                       'true_answer': true_answer,
                       'scenarios': scenarios.scenario,
                       'options': options}

df_prompts_rating = pd.DataFrame(dict_prompts_rating)
df_prompts_rating.to_csv("Maxims_prompts_Rating.csv", index=False)

## Generate prompts for free production

In [5]:
# Read the instructions for free production
file_path_free = 'raw_stimuli/MaximsInstructions_Free.txt'

with open(file_path_free, 'r') as file:
    instructions_free = file.read()


# Combine instructions and scenarios together
new_prompts = []
for i in range(len(scenarios.scenario)):
    new_prompts.append(instructions_free + " " + "Scenario: " + scenarios.scenario[i])

dict_prompts_free = {'item_number': scenarios.item_number,
                       'prompt': new_prompts,
                       'true_answer_key': [keys_items_pair[1] for keys_items_pair in keys_items_pairs],
                       'true_answer': true_answer,
                       'scenarios': scenarios.scenario,
                       'options': options}

df_prompts_free = pd.DataFrame(dict_prompts_free)
df_prompts_free.to_csv("Maxims_prompts_Free.csv", index=False)

